# Quantum Baked ETFs

<h2>Outline</h2>

- [Features selection](#Features-selection)
- [Dynamic Portfolio Optimization problem](#Dynamic-Portfolio-Optimization-problem)
- [Comparing quantum and classical portfolios](#Comparing-quantum-and-classical-portfolios)

<hr>
<h2>Dynamic Portfolio Optimization problem</h2>

This work is inspired from the paper: *Dynamic Portfolio Optimization with Real Datasets Using Quantum Processors and Quantum-Inspired Tensor Networks* (https://arxiv.org/abs/2007.00017). 

Our contributions include:  

- Using a feature selection tool for assets allocation during periodical rebalancing period
- Implement feature selection using a quantum annealer
- Comparing DWave processor with hybrid and classical solvers



In [1]:
import numpy as np
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import time

seed = 1000
np.random.seed(seed)

#### Importing Leap solvers

In [2]:
import dimod
from dimod import BinaryQuadraticModel
from dwave.system import LeapHybridSampler
from dwave.system import DWaveCliqueSampler
from neal import SimulatedAnnealingSampler
from dimod import ExactSolver

#### Deriving stocks prices from the SP 500 

In [68]:
def extract_stocks(num_stocks):
    df=pd.read_csv('snp_500_hist.csv',index_col='None') # The file 'snp_500_hist.csv' contains SP 500 data
    df['log_ret'] = 1.00
    df_scores=pd.read_csv('scores.csv',index_col=0)
    
    def log_ret(f,stock):
        f_new=f.reset_index()
        if f_new['ticker'].loc[0] == stock:
            for i in range(len(f_new['log_ret'])-1):
                x=f_new['closeadj'].loc[i+1]/f_new['closeadj'].loc[i] - 1  #f_new['closeadj'].loc[i]/f_new['closeadj'].loc[i-1] - 1
                f_new['log_ret'].loc[i]=x
            return f_new
        
    stocks=list(df_scores.index)[-num_stocks:] # Choose featured selected stocks
    
#     list_ticker_name=df['ticker'].unique()
#     stocks = np.random.choice(list_ticker_name, num_stocks) # Choose randomly num_stocks out of 505
    
    for i in range(len(stocks)):
        df0=df.groupby('ticker').apply(log_ret,stocks[i])
        if i==0: returns=pd.DataFrame(index=df0['date'], columns=stocks)
        returns[stocks[i]]=df0['log_ret'].to_list()
        
    return stocks, returns

#### Defining the portfolio

In [69]:
def portfolio(num_stocks, total_price, min_trading_days, num_trading_days):

    price_cap = 0.1*total_price # Maximum investment in each asset
    tmin = min_trading_days
    tmax = tmin + num_trading_days
    T = tmax-tmin # Total number of trading days
    
    stocks, returns = extract_stocks(num_stocks)
    
    log_returns = np.log(1.0 + returns)  #The assets forecast log-returns 
    
    # Computing the covariance matrix for each trading day
    cov = np.zeros((T, num_stocks, num_stocks)) 

    n = 0
    for t in range(tmin, tmax):
        cov[n,:,:] = np.cov( log_returns[0:t].T )    
        n += 1

    return log_returns, cov, stocks

[Back to Dynamic Portfolio Optimization problem](#Dynamic-Portfolio-Optimization-problem)

<hr>
<h2>Building the QUBO matrix</h2>

The dynamic portfolio optimization problem can be formulated as a classical Hamiltonian of the form: $ H = H_{1} + H_{2} + H_{3} + H_{4}$ where: 

$H_{1} = -\sum_t \mu_t^{T} w_{t}$ , 


$H_{2} = \frac{\gamma}{2} \sum_t w_t^{T} \Lambda_{t} w_{t}$ , 


$H_{3} = \lambda \sum_t {(\Delta w_t)}^{2} $ ,


$H_{4} = \rho \sum_t {(u^T w_t - 1)}^{2} $.



$\gamma$ is the *risk_aversion* and $\lambda$ is the *optimal coefficient for transaction costs*  and $\rho$ is Lagrange multiplier for normalization.



The QUBO matrix is obtained by recasting the Hamiltonian as $H = x^T Q x$, where $x \in {\{0,1\}}^{L}$ is binary encoding representation of a trading trajectory. For more details refer to the paper above. 

In [5]:
#Building the H1 QUBO matix 
def qubo1(num_bits, num_stocks, T, total_price, log_returns, stocks):
    
    L = num_bits * num_stocks * T
    QQ1 = np.zeros((L))
    
    for t in range(T):
        for n in range(num_stocks):
            for q in range(num_bits):
                i = (q + num_bits*n) + num_stocks*num_bits*t
                day=log_returns.index[t]
                stock=stocks[n]
                
                QQ1[i] = -2**q * log_returns[stock].loc[day]
                
    QQ1 = QQ1/total_price           
    Q1 = np.diag(QQ1)
    
    return Q1

In [6]:
#Building the H2 QUBO matix
def qubo2(num_bits, num_stocks, T, total_price, cov, risk_aversion):
    
    L = num_bits * num_stocks * T
    QQ2 = np.zeros((L, L))
    
    for t in range(T):
        for n in range(num_stocks):
            for q in range(num_bits):
                l = q + num_bits*n
                
                for m in range(num_stocks):
                    for qp in range(num_bits):
                        p = qp + num_bits*m
                        
                        irow = l + num_stocks*num_bits*t
                        icol = p + num_stocks*num_bits*t
                        
                        QQ2[irow, icol] = 2**(q+qp) * cov[t, n, m]
                        
    Q2 = 0.5*risk_aversion*QQ2/(total_price**2)
    
    return QQ2

In [7]:
#Building the H3 QUBO matix
def qubo3(num_bits, num_stocks, T, total_price, lambda_coefficient):
    
    L = num_bits * num_stocks * T
    
    encoding = np.array([2**q for q in np.arange(num_bits)]) # Binary encoding of the weights
    encoding_matrix = np.zeros((num_stocks*T, L))
    
    for n in range(num_stocks*T):
        left = n*num_bits
        right = (n+1)*num_bits
        encoding_matrix[n, left:right] = encoding
        
    difference_matrix = np.zeros((num_stocks*T, num_stocks*T))
    diff = np.concatenate((-np.identity(num_stocks), np.identity(num_stocks)), axis=1)

    for t in range(T-1):
        left = t*num_stocks
        right = left + 2*num_stocks 
        up = t*num_stocks
        down = (t+1)*num_stocks
        difference_matrix[up:down,left:right] = diff 
        
    QQ3 = np.dot(difference_matrix, encoding_matrix)
    Q3  = lambda_coefficient * np.dot(QQ3.T, QQ3)/(total_price**2)
    
    return Q3

In [8]:
#Building the H4 QUBO matix
def qubo4(num_bits, num_stocks, T, total_price, rho_multiplier):
    
    L = num_bits * num_stocks * T
    I0 = np.zeros((L, L))
    
    for t in range(T):
        for n in range(num_stocks):
            for q in range(num_bits):
                l = q + num_bits*n

                for m in range(num_stocks):
                    for qp in range(num_bits):
                        p = qp + num_bits*m

                        irow = l + num_stocks*num_bits*t
                        icol = p + num_stocks*num_bits*t
                    
                        I0[irow, icol] = 2**(q+qp)

    I = rho_multiplier*I0/(total_price**2)

    J0 = np.zeros((L))
    
    for t in range(T):
        for n in range(num_stocks):
            for q in range(num_bits):
                i = (q + num_bits*n) + num_stocks*num_bits*t
                
                J0[i] = -2**(q + 1)

    J0 = rho_multiplier*J0/total_price           
    J = np.diag(J0)
    Q4 = I + J
    
    return Q4

In [9]:
def qubo_matrix(num_stocks, num_bits, total_price, min_trading_days, num_trading_days, rho_multiplier, lambda_coefficient = 0.00001, risk_aversion=1.0):
    
    price_cap = 0.1*total_price # Maximum investment in each asset
    
    # Getting the log_returns and covariance matrix
    log_returns, cov, stocks = portfolio(num_stocks, total_price, min_trading_days, num_trading_days)
       
    # Buiding the different QUBO matrices
    Q1 = qubo1(num_bits, num_stocks, num_trading_days, total_price, log_returns, stocks)
    Q2 = qubo2(num_bits, num_stocks, num_trading_days, total_price, cov, risk_aversion)
    Q3 = qubo3(num_bits, num_stocks, num_trading_days, total_price, lambda_coefficient)
    Q4 = qubo4(num_bits, num_stocks, num_trading_days, total_price, rho_multiplier)
    
    Q = Q1 + Q2 + Q3 + Q4
    e_offset = rho_multiplier
    
    return Q, e_offset, stocks

[Back to Building the QUBO matrix](#Building-the-QUBO-matrix)

<hr>
<h2>Comparing quantum and classical portfolios</h2>

#### Weights encoding

In [10]:
def weights(x, num_stocks, num_bits, T, total_price):
    
    encoding = np.array([2**q for q in np.arange(num_bits)]) # Binary encoding of the weights
    weight = np.sum(x.reshape(T * num_stocks, num_bits) * encoding, axis=1)
    
    return weight/total_price

#### Sharpe Ratio and Transaction costs

In [11]:
def metrics(mu, cov, w, T, num_stocks, lambda_coefficient = 0.00001):
    
    returns = 0.0
    risk = 0.0
    deltaW = np.zeros((T-1, num_stocks))

    for t in range(T-1):
        returns += np.dot(mu[t, :], w.T[:, t]) 
        risk += np.dot( w[t, :], np.dot(cov[t,:,:], w.T[:, t]))
        deltaW[t, :] = (w[t+1, :] - w[t, :])**2

    returns += np.dot(mu[T-1, :], w.T[:, T-1])
    risk += np.dot( w[T-1, :], np.dot(cov[T-1,:,:], w.T[:, T-1]))

    sharperatio = returns / np.sqrt(0.5*risk)
    profits = returns - lambda_coefficient*np.sum(deltaW)

    return sharperatio, profits

### Comparison between Exact solver and DWave machine

In [22]:
list_num_stocks = [2] 
min_trading_days = 20
num_trading_days = 3
total_price = 10
price_cap = 0.1*total_price
exact_solver_results = [None] * len(list_num_stocks)
dwave_solver_results = [None] * len(list_num_stocks)
assets = [None] * len(list_num_stocks)

rho_multiplier = 0.5 

#Binary encoding of weights
num_bits = np.rint(np.log2(total_price + 1.0 )).astype(int)
    
for n, num_stocks in enumerate(list_num_stocks):
    
    Q, e_offset, assets[n] = qubo_matrix(num_stocks, num_bits, total_price, min_trading_days, num_trading_days, rho_multiplier)
    bqm = BinaryQuadraticModel.from_qubo(Q, offset= e_offset)
    
    solver = ExactSolver()
    exact_solver_results[n] = solver.sample(bqm)
    
    sampler =  DWaveCliqueSampler()
    dwave_solver_results[n] = sampler.sample(bqm, num_reads=500, annealing_time=1000, answer_mode='raw', return_embedding=True)

In [24]:

for n, num_stocks in enumerate(list_num_stocks):    
    
    logreturns, cov, _ = portfolio(num_stocks, total_price, min_trading_days, num_trading_days)
    mu = np.array(logreturns[min_trading_days:num_trading_days+min_trading_days])
    
    print("Results from the Exact Solver")
    configa = np.array(list(exact_solver_results[n].first.sample.values()))
    wa = weights(configa, num_stocks, num_bits, num_trading_days, total_price)
    wa = wa.reshape(num_trading_days, num_stocks)    
    exact_solver_sharpe_ratio, exact_solver_profits = metrics(mu, cov, wa, num_trading_days, num_stocks)
    
    print(f'\nThe {num_stocks} assets in the portfolio are: {assets[n]} and are traded for {num_trading_days} days.')
    print("The trading trajectory is :")
    print(wa) 
    print(f'The trajectory normalization is : {np.sum(wa, axis=1)} and the energy is {exact_solver_results[n].first.energy}') 
    print(f'Exact solver Sharpe ratio is {exact_solver_sharpe_ratio}, and the profits are {exact_solver_profits * 100}%')
    
    
    print("\n\nResults from the DWave Solver")
    configb = np.array(list(dwave_solver_results[n].first.sample.values()))
    wb = weights(configb, num_stocks, num_bits, num_trading_days, total_price)
    wb = wb.reshape(num_trading_days, num_stocks)
    dwave_solver_sharpe_ratio, dwave_solver_profits = metrics(mu, cov, wb, num_trading_days, num_stocks)
    
    print(f'\nThe {num_stocks} assets in the portfolio are: {assets[n]} and are traded for {num_trading_days} days.')
    print("The trading trajectory is :")
    print(wb)
    print(f'The trajectory normalization is : {np.sum(wb, axis=1)} and the energy is {dwave_solver_results[n].first.energy}') 
    print(f'Dwave solver Sharpe ratio is {dwave_solver_sharpe_ratio}, and the profits are {dwave_solver_profits *100}%')


Results from the Exact Solver

The 2 assets in the portfolio are: ['EMN', 'ZTS'] and are traded for 3 days.
The trading trajectory is :
[[0.3 0.7]
 [0.3 0.7]
 [0.4 0.6]]
The trajectory normalization is : [1. 1. 1.] and the energy is -0.9799341604033462
Exact solver Sharpe ratio is 0.35875315881651065, and the profits are 0.37180182292263614%


Results from the DWave Solver

The 2 assets in the portfolio are: ['EMN', 'ZTS'] and are traded for 3 days.
The trading trajectory is :
[[0.3 0.7]
 [0.3 0.7]
 [0.5 0.5]]
The trajectory normalization is : [1. 1. 1.] and the energy is -0.9797225638931327
Dwave solver Sharpe ratio is 0.6493758572635614, and the profits are 0.7062885375453511%


### Comparison between LeapHybrid Solver and Simulated Annealing

In [71]:
list_num_stocks = [100] #, 200, 450] 
min_trading_days = 20
num_trading_days = 100
total_price = 10 
#price_cap = 0.1* total_price
leap_hybrid_solver_results = [None] * len(list_num_stocks)
leap_hybrid_solver_times = [None] * len(list_num_stocks)
hybrid_solver_sharpe_ratio = [None] * len(list_num_stocks)
hybrid_solver_profits = [None] * len(list_num_stocks)

sa_solver_results = [None] * len(list_num_stocks)
sa_solver_times = [None] * len(list_num_stocks)
sa_solver_sharpe_ratio = [None] * len(list_num_stocks)
sa_solver_profits = [None] * len(list_num_stocks)

assets = [None] * len(list_num_stocks)

rho_multiplier = 1.0 

#Binary encoding of weights
num_bits = np.rint(np.log2(total_price + 1.0 )).astype(int)
    
for n, num_stocks in enumerate(list_num_stocks):
    
    Q, e_offset, assets[n] = qubo_matrix(num_stocks, num_bits, total_price, min_trading_days, num_trading_days, rho_multiplier)
    bqm = BinaryQuadraticModel.from_qubo(Q, offset= e_offset)
    
    start = time.time()
    solver = LeapHybridSampler()
    leap_hybrid_solver_results[n] = solver.sample(bqm)
    end = time.time()
    leap_hybrid_solver_times[n] = end-start
      
    start = time.time()
    sampler =   SimulatedAnnealingSampler()
    sa_solver_results[n] = sampler.sample(bqm, num_reads=100, num_sweeps=1000) #.truncate(5)
    end = time.time()
    sa_solver_times[n] = end-start
    

for n, num_stocks in enumerate(list_num_stocks): 

    logreturns, cov, _ = portfolio(num_stocks, total_price, min_trading_days, num_trading_days)
    mu = np.array(logreturns[min_trading_days:num_trading_days+min_trading_days])
    
    print("\nResults from the Hybrid Solver")
    config = np.array(list(leap_hybrid_solver_results[n].first.sample.values()))
    w = weights(config, num_stocks, num_bits, num_trading_days, total_price)
    w = w.reshape(num_trading_days, num_stocks)
    hybrid_solver_sharpe_ratio[n], hybrid_solver_profits[n] = metrics(mu, cov, w, num_trading_days, num_stocks)
      
    print(f'\nThe {num_stocks} assets in the portfolio are: {assets[n]} and are traded for {num_trading_days} days.')
    print("The trading trajectory is :")
    print(w) 
    print(f'The trajectory normalization is : {np.sum(w, axis=1)} and the energy is {leap_hybrid_solver_results[n].first.energy}') 
    print(f'The hybrid solver ran for {leap_hybrid_solver_times[n]} s.')
    print(f'Hybrid solver Sharpe ratio is {hybrid_solver_sharpe_ratio[n]}, and the profits are {hybrid_solver_profits[n]*100}%')
    
    print("\n\nResults from the SA Solver")
    config = np.array(list(sa_solver_results[n].first.sample.values()))
    w = weights(config, num_stocks, num_bits, num_trading_days, total_price)
    w = w.reshape(num_trading_days, num_stocks)
    sa_solver_sharpe_ratio[n], sa_solver_profits[n] = metrics(mu, cov, w, num_trading_days, num_stocks)
    
    print(f'\nThe {num_stocks} assets in the portfolio are: {assets[n]} and are traded for {num_trading_days} days.')
    print("The trading trajectory is :")
    print(w) 
    print(f'The trajectory normalization is : {np.sum(w, axis=1)} and the energy is {sa_solver_results[n].first.energy}') 
    print(f'The SA solver ran for {sa_solver_times[n]} s.')
    print(f'SA solver Sharpe ratio is {sa_solver_sharpe_ratio[n]}, and the profits are {sa_solver_profits[n]*100}%')


Results from the Hybrid Solver

The 100 assets in the portfolio are: ['AES', 'BDX', 'VRSK', 'ADM', 'VRTX', 'TDY', 'BIO', 'BR', 'ALGN', 'AJG', 'WEC', 'CAT', 'AVY', 'TYL', 'ITW', 'HII', 'FTNT', 'PENN', 'GOOGL', 'IPGP', 'INTU', 'LYB', 'PCAR', 'EL', 'GNRC', 'DXCM', 'ENPH', 'ETR', 'NVDA', 'NTAP', 'CME', 'MLM', 'IEX', 'IDXX', 'MNST', 'FOXA', 'ILMN', 'MCK', 'HPE', 'PKI', 'SNPS', 'CMS', 'CNC', 'CNP', 'JNJ', 'COO', 'ROST', 'ROL', 'PKG', 'CPRT', 'K', 'REGN', 'KHC', 'PWR', 'DFS', 'PNR', 'DTE', 'CRL', 'ICE', 'FAST', 'LEN', 'DHR', 'DRI', 'KO', 'TXN', 'MU', 'HOLX', 'SPGI', 'LH', 'INCY', 'ABT', 'ABMD', 'KLAC', 'ZBRA', 'WLTW', 'WY', 'UPS', 'AKAM', 'SWK', 'COG', 'ROK', 'TER', 'PH', 'PFE', 'BF.B', 'PEG', 'TTWO', 'HES', 'EIX', 'EMR', 'EOG', 'FB', 'APA', 'AON', 'FBHS', 'VIAC', 'GM', 'MCO', 'EMN', 'ZTS'] and are traded for 100 days.
The trading trajectory is :
[[0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.1 0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.3 ... 

In [53]:
parameters = [ list_num_stocks[i] * (num_bits * num_trading_days) for i in range(len(list_num_stocks))]

In [54]:
hybrid_name_dict1 = {
            'Number of stocks': list_num_stocks,
            'Sharpe ratio': hybrid_solver_sharpe_ratio,
            'Profits (in %)': [hybrid_solver_profits[i]*100 for i in range(len(hybrid_solver_profits))],
            'Runtimes (in s)': leap_hybrid_solver_times,
            'Number of parameters': parameters,
            'Hilbert space': [ 2**parameters[i] for i in range(len(parameters))]
          }

# hybrid_df = pd.DataFrame(hybrid_name_dict1)
# file_name = 'leap_hybrid_results1.csv'
# hybrid_df.to_csv(file_name, sep=",")
# hybrid_df

In [67]:
hybrid_df1=pd.read_csv('leap_hybrid_results.csv')
hybrid_df2 = pd.DataFrame(hybrid_name_dict1)

file_name = 'leap_hybrid_results1.csv'
hybrid_df2.to_csv(file_name, sep=",")

In [65]:
sa_name_dict = {
            'Number of stocks': list_num_stocks,
            'Sharpe ratio': sa_solver_sharpe_ratio,
            'Profits (in %)': [sa_solver_profits[i]*100 for i in range(len(sa_solver_profits))],
            'Runtimes (in s)': sa_solver_times,
            'Number of parameters': [ list_num_stocks[i] * (num_bits * num_trading_days) for i in range(len(list_num_stocks))],
            'Hilbert space': [ 2**parameters[i] for i in range(len(parameters))]
          }

sa_df = pd.DataFrame(sa_name_dict)

file_name = 'sa_results1.csv'
sa_df.to_csv(file_name, sep=",")
sa_df

,Number of stocks,Sharpe ratio,Profits (in %),Runtimes (in s),Number of parameters,Hilbert space
0,100,-0.158793,-0.087683,2.879799,300,0
1,200,0.697526,0.393605,11.240008,600,0
2,450,0.452607,0.379317,58.824439,1350,0


In [46]:
print("Geeks : %2d, Portal : %5.5f" % (1, 05.333))

Geeks :  1, Portal : 5.33300


In [47]:
[ 2**parameters[i] for i in range(len(parameters))]

[32768, 1073741824, 1152921504606846976, 0]

In [48]:
parameters

[15, 30, 60, 150]